## Explanations with LIME

Local Interpretable Model-agnostic Explanation ([LIME](https://arxiv.org/abs/1602.04938)) provides a fast and relatively simple method for `locally` explaining black box models. The LIME algorithm can be simplified into a few steps

- For a given data point, randomly perturb its features repeatedly. For tabular data, this entails adding a small amount of noise to each feature.  
- Get predictions for each perturbed data instance. This helps us build up a local picture of the decision surface at that point.
- Use predictions to compute an approximate linear "explanation model" using predictions. Coefficients of the linear model are used as explanations.


The LIME python library provides interfaces for explaining models built on tabular (TabularExplainer), image (LimeImageExplainer), and text data (LimeTextExplainer). 

In the following section, we will attempt to explain predictions from a single test data instance for all our trained models using the LimeTabularExplainer.


## LIME Tabular Explainer: Explain a test data instance for all models

In the following section, we will generate and visualize lime explanations for a given data point in our test set. We will do this for all our trained models.

In [ ]:
import pandas as pd
import re
import numpy as np

data = pd.read_pickle('data/global_train_data.pkl').sample(10000)
data = data.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
data = data.replace([np.inf, -np.inf], np.nan)

y = data['TARGET'].values
X = data.drop(['TARGET', 'SK_ID_CURR'], axis=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd


pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', StandardScaler())])
X_pre = pd.DataFrame(pipeline.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_pre, y, random_state=42)


params = {'boosting_type': 'gbdt', 'objective': 'binary', 'max_depth': 18,
          'n_jobs': -1, 'num_leaves': 30, 'learning_rate': 0.02, 'n_estimators': 1600,
          'max_bin': 512, 'subsample_for_bin': 200, 'subsample': 0.8,
          'subsample_freq': 1, 'colsample_bytree': 0.8,
          'reg_alpha': 80, 'reg_lambda': 20,
          'min_split_gain': 0.5, 'min_child_weight': 1,
          'min_child_samples': 10, 'scale_pos_weight': 11.5, 'num_class': 1,
          'metric': 'auc', 'learning_rate': 0.02
          }

LGB_clf = lgb.LGBMClassifier(**params)
LGB_clf.fit(X_train, y_train)

In [ ]:
# Read data and merge
df = pd.read_csv('data/application_train.csv')    
# Optional: Remove 4 applications with XNA CODE_GENDER (train set)
df = df[df['CODE_GENDER'] != 'XNA']
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

categorical_features = [idx for idx, col in enumerate(df.columns) if df[col].dtype == 'object']
categorical_names = {}
for idx in categorical_features:
    categorical_names[idx] = [df.columns[idx] + '_' + s for s in list(set(df[df.columns[idx]])) if not isinstance(s, float)] 

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

test_data_index = 10
top_x = 15

def get_lime_explainer(X_train, y_train, idx, classifier):
# explain first sample from test data
lime_explainer = LimeTabularExplainer(X_train,
                                      feature_names=list(X_train.columns),
                                      class_names=set(y_train),
                                      discretize_continuous=False,
                                      kernel_width=np.sqrt(
                                          len(X_train.columns)) * 0.75,
                                      mode="classification")

explanation = lime_explainer.explain_instance(
    X_test.to_numpy()[idx], LGB_clf.predict_proba, num_features=top_x)

ex_holder = {}
for feat_index, ex in explanation.as_map()[1]:
    ex_holder[list(X.columns)[feat_index]] = ex

actual_pred = LGB_clf.predict_proba(
    X_test.to_numpy()[test_data_index].reshape(1, -1))

perc_pred_diff = abs(actual_pred[0][1] - explanation.local_pred[0])

lime_metrics = []
lime_metrics.append({"lime class1": explanation.local_pred[0],
                     "actual class1": actual_pred[0][1],
                     "class_diff": round(perc_pred_diff, 3), "model": "LGB"})
# break
print(lime_metrics)
print(ex_holder)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

color_list =  sns.color_palette("dark", len(X.columns)) 

def plot_lime_exp(fig, fig_index, exp_data, title):
    features =  list(exp_data.keys())[::-1]
    explanations = list(exp_data.values())[::-1]
    ax = fig.add_subplot(fig_index) 
    lime_bar = ax.barh( features, explanations ) 
    ax.set_title(title, fontsize = 20)
    for i,bar in enumerate(lime_bar):
        bar.set_color(color_list[list(X.columns).index(features[i])])
        plt.box(False) 
fig = plt.figure(figsize=(19,8))

# Plot lime explanations for trained models
fig_index = int('231')
plot_lime_exp(fig, fig_index, ex_holder, "LGB")

plt.suptitle( " LIME Explanation for single test data instance.  Top " + str(top_x) + " Features", fontsize=20, fontweight="normal")
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

## Debugging  LIME: Should I trust the Explanation?


Underneath, the LIME algorithm uses an approximate linear model to derive local explanations. Like any other ML model, this explanation model can have *issues* too. So, what can we do to build   confidence in the quality of an explanation. As a first step, we can check if the local model is indeed a good approximator for the original model.    

In [ ]:
# Plot run time for explanations
lime_metrics_df = pd.DataFrame(lime_metrics)  
lime_metrics_df_ax = lime_metrics_df[["lime class1", "actual class1", "model"]].plot(kind="line", x="model", title="LIME Actual Prediction vs Local Prediction ", figsize=(22,6))
lime_metrics_df_ax.title.set_size(20)
lime_metrics_df_ax.legend(["Lime Local Prediction", "Actual Prediction"])
plt.box(False)

The plot above shows the predictions made by the LIME local model and the original model for the explained data instance. Both numbers should be close.  When they are not, this may raise questions as to if we can trust the explanation. There are few things that can be done:
-  Modify the parameters of LIME to yield a better explanation. E.g. increase the number of perturbations (LIME ) or kernel width (see related discussion  [here](https://github.com/marcotcr/lime/issues/209)), 
- Improve our original model (In this case, we know that the Decision Tree shows signs of overfitting). 

The next explanation method we will consider (SHAP) aims to address such inconsistencies. Let's go!

## Explanations with SHAP

To provide some intuition on how SHAP works, consider the following scenario. We have a group of data scientists (Sarah, Jessica, and Patrick) who collaborate to build a great predictive model for their company. At the end of the year, their efforts result in an increase in profit of which $5 million must now be shared amongst our 3 heroes. Assuming we have good ways to measure (or simulate) the contributions of each data scientist, how can we allocate this profit such that each person is fairly rewarded commensurate to their actual contribution?


[Shapley values](https://en.wikipedia.org/wiki/Shapley_value) provide a method for this specific type of allocation (collaborative multiplayer game setting) with a set of desirable axiomatic properties (Efficiency, Symmetry, Linearity, Anonymity, Marginalism) that guarantee fairness. These values are computed by computing the average marginal contribution of each person across all possible orderings. For example, imagine we assign only Sarah to the project and note the increase in profit (their marginal contribution). We then add Jessica and note the corresponding increase. We then add Patrick and note their contribution. This is repeated for all possible orderings (e.g. {Patrick, Jessica, Sarah}, {Jessica, Sarah, Patrick}, etc ) and the average marginal contribution for each person is computed.
Extending this to machine learning, we can think of each feature as comparable to our data scientists and the model prediction as the profits. To explain our model, we repeatedly add each feature and note its marginal contribution to model prediction. Importantly, we want to use the Shapley values to assign credit to each feature, because they provide two important guarantees (e.g., LIME, Feature Permutation, Feature Importance) that other methods do not provide:


- local accuracy (an approximate model used to explain the original model should match the output of the original model for a given input)
- consistency (if the original model changes such that a feature has a larger impact in every possible ordering, then its attribution should not decrease)



In practice, a few simplifications are required to compute Shapley values. Perhaps the most important is related to how we simulate the adding or removal of features while computing model prediction. This is challenging because there is no straightforward way to "remove" a feature for most predictive models at test time. We can either replace the feature with its mean value, median value. In the SHAP library implementation, a “missing” feature is simulated by replacing the feature with the values it takes in the background dataset. 


## The SHAP Library Implementation.

The SHAP library contains implementations for several types of explanations that leverage Shapley values. These include the `TreeExplainer` which is optimized (and fast) for tree based models; `DeepExplainer` and `GradientExplainer` for neural networks; and  `KernelExplainer`, which makes no assumptions about the underlying model to be explained (model agnostic like LIME).

To explain a model on a test set using `KernelExplainer`, the SHAP library api is as follows:

```
import shap
explainer = shap.KernelExplainer(model.predict_proba, background_data)
shap_values = explainer.shap_values(X_test)
```

To unpack and understand the results from SHAP KernelExplainer, there are a few terms worth clarifying.


| Variable 	| Description 	|
|--------------------------	|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| model 	| the model to be explained 	|
| background_data 	| This is a required argument to KernelExplainer. Since most models aren’t designed to handle arbitrary missing data at test time, SHAP simulates a “missing” feature by replacing it with the values it takes in the background dataset. For small problems, this background dataset can be the whole training set, but for larger problems, it is suggested that a subsample of the training set (or the kmeans function to summarize the dataset) is used. Background data is optional for tree-based models. 	|
| explainer.expected_value 	| This is a field in the explainer object is displayed as the baseline in a SHAP force plot. It should be the same as the mean of the model output over the background dataset. One simple task which I found to be useful is to manually compute the mean prediction on the background dataset and see how it corresponds to the expected value output by SHAP. 	|
| shap_values 	| The shap_values returned by the explainer object are a measure of how each feature contributes to the difference between the model’s expected value and the prediction for that instance. The units of the Shapley values are in the units of the target variable. The sum of the shap values should be equal to the difference between the base value and the model prediction. 	|




## Kernel Explainer



In [ ]:
import shap

subsampled_train_data = shap.sample(X_train, 600, random_state=0) # use 600 samples of train data as background data
sample = X_test.to_numpy()[test_data_index].reshape(1,-1)
explainer = shap.KernelExplainer(LGB_clf.predict_proba, subsampled_train_data)
shap_values = explainer.shap_values(sample,  l1_reg="aic")

# explain first sample from test data
print("Kernel Explainer SHAP run time", round(elapsed_time,3) , " seconds. ", "LGB")
print("SHAP expected value", explainer.expected_value)
print("Model mean value", LGB_clf.predict_proba(X_train).mean(axis=0))
print("Model prediction for test data", LGB_clf.predict_proba(subsampled_test_data))
shap.initjs()
pred_ind = 0
shap.force_plot(explainer.expected_value[1], shap_values[1][0], subsampled_test_data[0], feature_names=X_train.columns)

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, subsampled_test_data, feature_names=X_train.columns, max_display=10)

## Tree Explainer



TreeExplainer on random forests takes about 0.091 seconds. This is compared to > 12.15 seconds required by KernelExplainer ( > **100x** faster).

In [ ]:

sample = X_test.to_numpy()[test_data_index].reshape(1,-1)

# explain first sample from test data
explainer = shap.TreeExplainer(LGB_clf)
shap_values = explainer.shap_values(sample)

print("Tree Explainer SHAP run time", round(elapsed_time,3) , " seconds. ", "LGB")
print("SHAP expected value", explainer.expected_value)
print("Model mean value", LGB_clf.predict_proba(X_train).mean(axis=0))
print("Model prediction for test data", LGB_clf.predict_proba(sample))
shap.initjs()
pred_ind = 0
shap.force_plot(explainer.expected_value[1], shap_values[1][0], sample[0], feature_names=X_train.columns)

## Kernel Explainer - Explain a test data instance for all models

In the following section, we will generate and visualize explanations using the SHAP KernelExplainer for a given data point in our test set (same as we did for LIME). 

In [ ]:
import shap

def get_kernel_shap_explainer(model, background_data):  
    shap_explainer = shap.KernelExplainer(model.predict_proba, background_data)   
    return shap_explainer 

def shap_explain(explainer, test_data): 
    shap_values = explainer.shap_values(test_data, l1_reg="aic")

    return shap_values

shap_data_explainations = []
shape_explanation_time = []
feat_names = list(X_train.columns) 
data_subsample = 500 

subsampled_train_data = shap.sample(X_train, 600, random_state=0) # use 600 samples of train data as background data

sampled_scaled_train_data = shap.sample(subsampled_train_data, data_subsample) # subsample background data to make things faster

shap_explainer  = get_kernel_shap_explainer(LGB_clf, subsampled_train_data)

# explain first sample from test data 
sample = X_test.to_numpy()[test_data_index].reshape(1,-1)
shap_values = shap_explain(shap_explainer, sample) 
idx = np.argsort(np.abs(shap_values[1][0]))[::-1] 
ex_holder = { feat_names[idx[i]] : shap_values[1][0][idx[i]] for i in range(top_x)} 

shap_data_explainations.append(ex_holder) 
shape_explanation_time.append({"time": elapsed_time, "model": "LGB" })


In [ ]:
def plot_shap_exp(fig, fig_index, exp_data, title):
    features =  list(exp_data.keys())[::-1]
    explanations = list(exp_data.values())[::-1]
    ax = fig.add_subplot(fig_index) 
    lime_bar = ax.barh( features, explanations ) 
    ax.set_title(title, fontsize = 20)
    for i,bar in enumerate(lime_bar):
        bar.set_color(color_list[list(current_data.columns).index(features[i])])
        plt.box(False) 


# Plot SHAP explanations for a given test set item
fig = plt.figure(figsize=(19,8))
for i, dex in enumerate(shap_data_explainations):
    fig_index = int('231')
    plot_lime_exp(fig, fig_index, shap_data_explainations[i], "LGB")

plt.suptitle( "Kernel SHAP Explanation for single test data instance.  Top " + str(top_x) + " Features", fontsize=20, fontweight="normal")
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

## Debugging SHAP KernelExplainer: Small background Dataset may lead to inconsistent expected value.

The process used by SHAP to simulate missing values draws from a background dataset (usually the training set). When the training dataset is large, the authors of SHAP suggest the use of a subsample of the training set as background data to reduce computation costs.  A side effect of using a small sub sample is that the expected value used by SHAP may differ from the model's mean value. The plots below compare the SHAP expected value and model mean prediction on training set for multiple background sample sizes.

- Run time increases (linearly) with increase in size of background dataset
- Expected value is closer to mean value on enter train set as the size of background subsample increases. (ideally these numbers should be the same. See relevant  [discussion on Github](https://github.com/slundberg/shap/issues/352))

The above results raise some questions as to the quality of explanations when a subsample is used as background data for SHAP. It is important to note that the TreeExplainer sidesteps this issue as it does not require a background dataset. Per the SHAP documentation

```
This argument (background data) is optional when feature_perturbation=”tree_path_dependent”, since in that case we can use the number of training samples that went down each tree path as our background dataset (this is recorded in the model object).
```

In [ ]:
current_model = trained_models[3]
clf = current_model["model"]["clf"]
scaler = current_model["model"]["scaler"]

sample_sizes = [50, 100, 150, 200, 300, 600, 1000, 2000, 3000, 4000]
metric_holder = []
 

for sample_size in sample_sizes:
  scaled_train_data = scaler.transform(X_train)
  sub_sampled_train_data = shap.sample(scaled_train_data, sample_size, random_state=0) # use x samples of train data as background data

  scaled_test_data = scaler.transform(X_test)
  test_data_index = 10
  subsampled_test_data =scaled_test_data[test_data_index].reshape(1,-1)

  start_time = time.time()
  explainer = shap.KernelExplainer(clf.predict_proba, sub_sampled_train_data)
  shap_values = explainer.shap_values(subsampled_test_data,  l1_reg="aic")
  elapsed_time = time.time() - start_time

  metric_holder.append({"class0 exp": explainer.expected_value[0], 
                        "class1 exp":explainer.expected_value[1],
                        "run time": elapsed_time,
                        "sample size": sample_size
                        })
   
metric_df = pd.DataFrame(metric_holder)


In [ ]:
mean_pred = clf.predict_proba(scaled_train_data).mean(axis=0)
metric_df["mean prediction class0"] = mean_pred[0]
metric_df["mean prediction class1"] = mean_pred[1]
metric_df.head()
setup_plot()
metrix_ax = metric_df[["sample size", "run time"]].plot(kind="line", x="sample size", title="Kernel SHAP Run time (seconds) vs Background Data Size", figsize=(22,4))
metrix_ax.title.set_size(20) 
plt.box(False)

setup_plot()
metrix_ax = metric_df[["sample size", "mean prediction class0", "class0 exp"]].plot(kind="line", x="sample size", title="Expected Value vs Mean Prediction", figsize=(22,4))
metrix_ax.title.set_size(20) 
metrix_ax.legend(["Mean prediction on train set", "SHAP Expected Value"])
plt.box(False)

## LIME vs SHAP : When to use what?

LIME and SHAP are both good methods for explaining models. In theory, SHAP is the better approach as it provides mathematical guarantees for the accuracy and consistency of explanations. In practice, the model agnostic implementation of SHAP (KernelExplainer) is slow, even with approximations. This speed issue is of much less concern if you are using a tree based model and can take advantage of the optimizations implemented in SHAP TreeExplainer (we saw it could be up to 100x faster than KernelExplainer).

Some additional limitations of both methods are mentioned below

- LIME is not designed to work with one hot encoded data. Considering that each data point is perturbed to create the approximate model, perturbing a one hot encoded variable may result in unexpected (meaningless) features. See discussion [here](https://github.com/marcotcr/lime/issues/153)
- LIME depends on the ability to perturb samples in meaningful ways. This perturbation is use case specific. E.g., for tabular data, this entails adding random noise to each feature; for images, this entails replacing superpixels within the image with some mean value or zeroes; for text, this entails removing words from the text. It is often useful to think through any side effects of these perturbation strategies with respect to your data to further build trust in the explanation.
- In some cases, the local model built by LIME may fail to approximate the behaviour of the original model. It is good practice to check for such inconsistencies before trusting LIME explanations. 

- LIME works with models that output probabilities for classification problems. Models like SVMs are not particularly designed to output probabilities (though they can be coerced into this with some [issues](https://scikit-learn.org/stable/modules/svm.html).). This may introduce some bias into the explanations.

- SHAP depends on background datasets to infer a baseline/expected value. For large datasets, it is computationally expensive to use the entire dataset and we have to rely on appromixations (e.g. subsample the data). This has implications for the accuracy of the explanation.

- SHAP explains the deviation of a prediction from the expected/baseline value which is estimated using the training dataset. Depending on the specific use case, it may be more meaningful to compute the expected value using a specific subset of the training set as opposed to the entire training set. For example, it may be more meaningful to explain a churn prediction with respect to how it deviates from customers who did not churn. Here, we might want to use the dataset of customers who did not churn as our background dataset.  See this issue [here](https://github.com/slundberg/shap/issues/435).

## Conclusion and Further Reading 

In this notebook, we have looked at LIME and SHAP. Actually, we have looked at just subsets of both of these remarkable tools, focusing on explaining models built on tabular data.  It is worth noting that  these methods can be extended to work on text, and image data as well. For additional reading on other interpretability methods, the [Interpretable ML book by  Christoph Molnar  is recommended](https://christophm.github.io/interpretable-ml-book/).

- Interpretability - Cloudera Fast Forward Labs https://ff06-2020.fastforwardlabs.com/
- SHAP vs Integrated Gradients https://blog.fiddler.ai/2019/08/should-you-explain-your-predictions-with-shap-or-ig/
- Interpretable Machine Learning https://christophm.github.io/interpretable-ml-book/
